In [1]:
!pip install -q datasets accelerate evaluate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# login vào huggingface để lấy dữ liệu đã tải lên
from huggingface_hub import login
login()

In [3]:
# login vào wandb để chuẩn bị theo dõi dự án
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import torch
torch.cuda.empty_cache()
# đảm bảo đang sử dụng GPU
import torch
if torch.cuda.is_available():
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find("failed") >= 0:
    print("Not connected to a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")

  # lấy tên GPu
  gpu_name = !nvidia-smi --query-gpu=gpu_name --format=csv
  gpu_name = gpu_name[1]
  GPU_NAME = gpu_name.replace(" ", "_") # remove underscores for easier saving
  print(f'GPU name: {GPU_NAME}')

  # Get GPU capability score
  GPU_SCORE = torch.cuda.get_device_capability()
  print(f"GPU capability score: {GPU_SCORE}")
  if GPU_SCORE >= (8, 0):
    print(f"GPU score higher than or equal to (8, 0), PyTorch 2.x speedup features available.")
  else:
    print(f"GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).")

  # In thông tin GPU
  print(f"GPU information:\n{gpu_info}")

else:
  print("PyTorch couldn't find a GPU, to leverage the best of PyTorch 2.0, you should connect to a GPU.")
if GPU_SCORE >= (8, 0):
  print(f"[INFO] Using GPU with score: {GPU_SCORE}, enabling TensorFloat32 (TF32) computing (faster on new GPUs)")
  torch.backends.cuda.matmul.allow_tf32 = True
else:
  print(f"[INFO] Using GPU with score: {GPU_SCORE}, TensorFloat32 (TF32) not available, to use it you need a GPU with score >= (8, 0)")
  torch.backends.cuda.matmul.allow_tf32 = False

GPU name: Tesla_T4
GPU capability score: (7, 5)
GPU score lower than (8, 0), PyTorch 2.x speedup features will be limited (PyTorch 2.x speedups happen most on newer GPUs).
GPU information:
Wed Dec 25 04:13:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              12W /  70W |      3MiB / 153

In [5]:
# lấy dữ liệu từ huggingface
from datasets import load_dataset
images = load_dataset("vieanh/sports_img_classification") # lấy link từ huggingface

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/629 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/288M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/279M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6655 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1664 [00:00<?, ? examples/s]

In [6]:
# ánh xạ nhãn (tự động lấy nhãn từ dữ liệu tổ chức theo định dạng imagefolder)
labels = images["train"].features["label"].names
print("Danh sách nhãn:", labels)

label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

Danh sách nhãn: ['badminton', 'cricket', 'football', 'karate', 'swimming', 'tennis', 'wrestling']


In [7]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [8]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

# chuẩn hóa ảnh theo các thông số của image_processor
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [9]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [10]:
transformed_images = images.with_transform(transforms)

In [11]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [12]:
# sử dụng thư viện đánh giá evaluate
import evaluate
metrics = evaluate.combine(
    ["accuracy", "f1","recall","precision"], force_prefix=False
)

In [13]:
#hàm tính toán Metrics
#phương pháp trung bình binary chỉ phù hợp với bài toán phân loại nhị phân (binary classification)
#import numpy as np

#def compute_metrics(p):
#    predictions, labels = p
#    predictions = np.argmax(predictions, axis=1)
#    outputs = metrics.compute(predictions=predictions,references=labels)
#    return outputs

# hàm tính toán Metrics phù hợp với bài toán phân loại đa lớp (multiclass)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    predictions = p.predictions.argmax(-1)  # Lấy nhãn dự đoán
    labels = p.label_ids                   # Lấy nhãn thực tế

    # Tính toán các chỉ số
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="weighted"  # Dùng "weighted" cho đa lớp
    )
    acc = accuracy_score(labels, predictions)

    # Trả về kết quả
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [14]:
# tạo model sử dụng AutoModelForImageClassification
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Image.MAX_IMAGE_PIXELS: Giới hạn kích thước tối đa của hình ảnh mà thư viện Pillow (PIL) có thể xử lý.
# Khi giá trị quá nhỏ, bạn có thể gặp lỗi DecompressionBombError nếu xử lý hình ảnh có kích thước lớn.
# None: Vô hiệu hóa giới hạn này, cho phép xử lý hình ảnh với bất kỳ kích thước nào.
from PIL import Image
Image.MAX_IMAGE_PIXELS = None  # hoặc để giá trị cực lớn như 933120000

In [16]:
training_args = TrainingArguments(
    f"vit-sports-cls",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=True,
    report_to="wandb",
)

# cấu hình Trainer
trainer = Trainer(
    # mô hình ViT đã được cấu hình bằng AutoModelForImageClassification
    model = model,

    # Các tham số huấn luyện như số epoch, batch size, learning rate, v.v.
    args=training_args,

    # Bộ xử lý dữ liệu (collate function) để chuẩn bị batch trong quá trình huấn luyện.
    data_collator=data_collator,

    train_dataset=transformed_images["train"],
    eval_dataset=transformed_images["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,

    # Sử dụng kỹ thuật dừng sớm (early stopping).
    # Nếu mô hình không cải thiện sau 3 epoch liên tiếp, quá trình huấn luyện sẽ dừng.
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

<ipython-input-16-be6c4b1165f0>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
%env WANDB_PROJECT=sports_img_classification
%env WANDB_LOG_MODEL=checkpoint

env: WANDB_PROJECT=sports_img_classification
env: WANDB_LOG_MODEL=checkpoint


In [20]:
# Bắt đầu quá trình huấn luyện mô hình
trainer.train()
# Kết thúc phiên làm việc với Weights & Biases sau khi huấn luyện hoàn tất
wandb.finish()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.171000,0.172852,0.948918,0.949258,0.948918,0.948893
2,0.097900,0.135628,0.958534,0.959698,0.958534,0.958253
3,0.040800,0.118449,0.956130,0.957052,0.956130,0.956082
4,0.070300,0.089200,0.969952,0.970078,0.969952,0.969907
5,0.137500,0.102929,0.968149,0.968347,0.968149,0.968168
6,0.006100,0.107263,0.968149,0.968801,0.968149,0.968194
7,0.008300,0.079484,0.969952,0.970131,0.969952,0.970010
8,0.007900,0.075374,0.981370,0.981616,0.981370,0.981391
9,0.059400,0.071397,0.975361,0.975616,0.975361,0.975405
10,0.039100,0.083812,0.974159,0.974285,0.974159,0.974147


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
No files have been modified since last commit. Skipping to prevent empty commit.


eval/accuracy,▁▃▃▆▅▅▆█▇▆
eval/f1,▁▃▃▆▅▅▆█▇▆
eval/loss,█▅▄▂▃▃▂▁▁▂
eval/precision,▁▃▃▆▅▅▆█▇▆
eval/recall,▁▃▃▆▅▅▆█▇▆
eval/runtime,▆▁▁▃▁▂▇▂▂█
eval/samples_per_second,▃█▇▆█▇▂▇▇▁
eval/steps_per_second,▃██▆█▇▂▇▇▁
train/epoch,▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▃▃▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇████
train/grad_norm,█▄▄▅▄▃▃▃▆▄▆▂▄▃▄▃▅▃▂▂▄▄▂▅▅▂▂▂▄▅▄▁▁▁▂▅▃▄▃▂


In [25]:
trainer.save_model()

In [26]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/vieanh/vit-sports-cls/commit/07e3386e98e7dabaf941989ec117372903e41217', commit_message='End of training', commit_description='', oid='07e3386e98e7dabaf941989ec117372903e41217', pr_url=None, repo_url=RepoUrl('https://huggingface.co/vieanh/vit-sports-cls', endpoint='https://huggingface.co', repo_type='model', repo_id='vieanh/vit-sports-cls'), pr_revision=None, pr_num=None)